In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#conda install Basemap
#from mpl_toolkits.basemap import Basemap
from itertools import combinations
pd.set_option('display.max_columns', 20000)
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
eq = pd.read_csv('data/03_equipment.csv', sep = ',"')
data = pd.read_csv('data/02_Data_test.csv', sep = ';')
positive_labels = pd.read_csv('data/01_Facts.csv', sep = ";", header = None)

/Users/natali/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Check if we have all data point found in positive labels in train set.   
We don't have any information about this number 158528852857, 158530004641 in the train data, however it is in positive labels.

In [4]:
to_delete = []
for i, row in enumerate(positive_labels.values):
    if len(data[data['msisdn'] == row[0]]) == 0:
        to_delete.append(row[0])
    if len(data[data['msisdn'] == row[1]]) == 0:
        to_delete.append(row[1])

In [5]:
positive_labels = positive_labels.drop(positive_labels[positive_labels[1].isin(to_delete)].index[0], axis = 0).reset_index(drop = True)
positive_labels = positive_labels.drop(positive_labels[positive_labels[1].isin(to_delete)].index[0], axis = 0).reset_index(drop = True)

Prettify ef file

In [10]:
eq.columns = ['tac', 'vendor', 'platform', 'type']
eq['tac'] = eq['tac'].apply(lambda x: x[1:])
eq['vendor'] = eq['vendor'].apply(lambda x: x[1:-2])
eq['platform'] = eq['platform'].apply(lambda x: x[1:-2])
eq['type'] = eq['type'].apply(lambda x: x[1:-3])

In [11]:
eq.head()

,tac,vendor,platform,type
0,00100429,PROTO,Proprietary,Phone
1,00100430,PROTO,Proprietary,Phone
2,00100431,PROTO,Proprietary,Phone
3,00100432,PROTO,Proprietary,Phone
4,00100433,PROTO,Proprietary,Phone


In [12]:
data.head()

,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,5029,40798,158529599791,3.531111e+14,0,1369252800974,37.9305,55.7815,2101,OUTDOOR,220.0,280.0
1,7782,56870,158520145943,1.292900e+13,5,1369252801396,37.4239,55.7426,2,OUTDOOR,45.0,105.0
2,7794,32226,158521798391,3.519940e+14,5,1369252802013,37.7034,55.5803,1125,OUTDOOR,0.0,60.0
3,7758,33528,158537830573,3.586270e+14,0,1369252802331,37.7895,55.7042,1158,OUTDOOR,210.0,270.0
4,5060,17568,158510204039,3.524581e+14,8,1369252803586,37.1900,55.8308,500,OUTDOOR,180.0,240.0


In [14]:
print("How many observations in our data set? -  {}".format(len(data)))
print("How many unique numbers (msisdn) in our data set? - {}".format(data['msisdn'].nunique()))

How many observations in our data set? -  174123
How many unique numbers (msisdn) in our data set? - 2361


It's computationally expensive to make all possible pair on current train data. Let's try to make data set smaller (include onluy data points which in positive_labels file) and see what result we can get from smaller data set.

In [16]:
list_of_positive_labels = []
for j in range(2):
    for i in range(len(positive_labels)):
        list_of_positive_labels.append(positive_labels.loc[i, j])

In [17]:
data_small = pd.DataFrame(columns = data.columns)
for i in list(data['msisdn'].unique()):
    if i in list_of_positive_labels:
        data_small = data_small.append(data[data['msisdn'] == i])
data_small = data_small.reset_index(drop = True)

"Msisdn" column is how many milliseconds passed from 1970-01-01. The next function convert this column in ordinary date-time format

In [20]:
data_small['tstamp'] = data_small['tstamp'].apply(lambda x: datetime.datetime(1970, 1, 1, 0, 0) + datetime.timedelta(milliseconds = x))

In [21]:
data_small.tstamp = data_small.tstamp.apply(lambda x: x.replace(second=0, microsecond=0))

Check if a person with two sim-card can have only one phone number (imei)

In [ ]:
count = 0
for i, row in enumerate(positive_labels.values):
    imei_1 = data_small[data_small['msisdn'] == positive_labels.loc[i,:][0]]['msisdn'].iloc[0]
    imei_2 = data_small[data_small['msisdn'] == positive_labels.loc[i,:][1]]['msisdn'].iloc[0]
    if imei_1 == imei_2:
        count += 1
        
if count = 0:
    print('False')

Since cid - identificator of base station unique in terms of lac the folowing cell creates columns 'lac_cid' as a combination of cid and lac and this gives as unique location for each station

In [25]:
data_small['lac_cid'] = data_small['lac'].astype(str)  + data_small['cid'].astype(str)

Grouping by msisdn. The first column of result dataframe msisdn itself, the second column - list of all locations where this number was registered

In [26]:
group = data_small.groupby(['msisdn'], as_index=False).agg({'lac_cid':(lambda x: list(x))}).rename(columns={'msisdn':'number', 'lac_cid':'set_'})

Creating all posible combinations of msisdn

In [27]:
df = pd.DataFrame(columns = ['number_1', 'set_1', 'number_2', 'set_2'])

for index in list(combinations(group.index, 2)):
    
    df0 = pd.DataFrame(group.loc[index[0]]).T.reset_index(drop=True)
    df1 = pd.DataFrame(group.loc[index[1]]).T.reset_index(drop=True)
    df_temp = pd.concat([df0, df1], axis = 1)
    df_temp.columns = ['number_1', 'set_1', 'number_2', 'set_2']
    df = df.append(df_temp)
df = df.set_index(pd.Index(range(len(df))))

The first assumption is that if user holds 2 numbers the locations rigistered by base station of exactly these numbers mainly intersect

In [35]:
def intersect_loc(x):
    list1 = x['set_1']
    list2 = x['set_2']
    
    min_len = min(len(list1), len(list2))
    res = []
    list_1 = list1.copy()
    for i in list_1:
        if i in list2:
            res.append(i)
            list1.remove(i)
            list2.remove(i)       
    return len(res)/(min_len+1)

In [36]:
df['intersect_loc'] = df.apply(intersect_loc, axis =1)

The following finction returns the dictionary of possible numbers which belong to the same person. It chooses candidates by selecting maximum intersection between them

In [43]:
def intersectin_time_loc(x):
    
    margin = datetime.timedelta(days=2)
    index_1 = x['number_1']
    index_2 = x['number_2']
    
    counter = 0
    set_1 = data_small[data_small['msisdn'] == index_1]
    set_2 = data_small[data_small['msisdn'] == index_2]
    time1, time2 = [], []
    for i in set_1['tstamp']:
        for j in set_2['tstamp']:
            if i - margin <= j <= i + margin:
                if set_1[set_1['tstamp'] == i]['lac'].iloc[0] == set_2[set_2['tstamp'] == j]['lac'].iloc[0]:
                    time1.append(i)
                    time2.append(j)
                    counter += 1
    return counter

In [52]:
df['intersectin_time_loc'] = df.apply(intersectin_time_loc, axis = 1)

In [ ]:
df_for_algo = df[['number_1', 'number_2', 'inter', 'intersectin_time_loc']]

In [ ]:
new_target = []

for entry in df_for_algo.values:
    for pair in positive_labels.values:
        add=0
        if entry[0] in pair and entry[1] in pair:
            new_target.append(1)
            add=1
            break
    if add==0:
        new_target.append(0)

df_for_algo['target'] = new_target

In [ ]:
ax = sns.scatterplot(x="inter", y="intersectin_time_loc", hue="target", data=df_for_algo)

In [ ]:
df_for_algo.to_csv('df_for_algo.csv')

In [ ]:
data_small['tac'] = data_small['imei'].apply(lambda x: str(x)[0:8])